In [62]:
import glob
import numpy as np
import math
from pyhdf.SD import SD, SDC
from __future__ import print_function
import matplotlib
matplotlib.use('TkAgg')
import pylab
import matplotlib.pyplot as plt
import datetime
import subprocess
from numpy import genfromtxt
import plotly.graph_objs as go
import plotly
from datetime import datetime
from scipy.spatial import cKDTree
#from IPython.display import clear_output

plotly.offline.init_notebook_mode(connected=True)

LatLongtree = []
NDVISTACK = []
NDVIaverage = []
sorteddates = []
dates = []

def initialize():
	global  NDVISTACK,NDVIaverage, sorteddates, dates, LatLongtree
	latitude = genfromtxt('/home/kumarv/tayal007/latitide.csv', delimiter=',')
	longitude = genfromtxt('/home/kumarv/tayal007/longitude.csv', delimiter=',')
	NDVISTACK = np.load('/home/kumarv/tayal007/NDVISTACK.npy')
	NDVIaverage = np.load('/home/kumarv/tayal007/NDVIaverage.npy')
	directorypath = glob.glob("/panfs/roc/groups/6/kumarv/tayal007/MODIS_h20v08/*.hdf");
	sorteddates = sortedtimestamp(directorypath)
	for k, v in sorted(sorteddates.items()):
    		dates.append(k);
	temp = []	
	for i in range(2400):
		for j in range(2400):
			tup = latitude[i,j], longitude[i,j]
			temp.append(tup)
	LatLongdata = np.array(temp)
	temp = None # clear memory
	latitude = None # clear memory
	longitude = None  # clear memory
	LatLongtree = cKDTree(LatLongdata)
	return ""


def getMatrixdimension(latitude, longitude):
	dists, indexed = LatLongtree.query(np.array([latitude,longitude]), k = 1)
	quotient = indexed//2400;
	remainder = indexed%2400;
	return (quotient,remainder -1)



def sortedtimestamp(directorypath):
	timestamps = dict()
	for path in directorypath:
            timestamps[datetime.datetime.strptime(path.split('.')[1][3:], '%y%j').date()] = path.split('.')[1]
	return timestamps;


def renderplot():
	plt.imshow(NDVIaverage)
	while True:
    		x = plt.ginput(1)
    		clear_output()
		#print (NDVISTACK[int(round(x[0][1])),int(round(x[0][0])),:])
    		try:
        		lat = latitude[int(round(x[0][0]))][int(round(x[0][1]))]
        		long = longitude[int(round(x[0][0]))][int(round(x[0][1]))]
        		print ('http://maps.google.com/?q=' + str(lat) + ',' + str(long))
        		showtimeseries(int(round(x[0][0])), int(round(x[0][1])));
    		except IndexError:
        		print ()


def showtimeseries(yvalue, xvalue):  
	counts = NDVISTACK[xvalue,yvalue,:]
	data = [go.Scatter(x=dates, y=counts)]
	plotly.offline.iplot(data)

In [3]:
# import webbrowser
# webbrowser.open('http://maps.google.com/?q=' + str(latitude[0][0]) + ',' + str(longitude[0][0]) , new=2)

True

In [13]:
import subprocess
def getDigitalglobe(latitude,longitude):
    # lat_min = float(sys.argv[1])
    # lon_min = float(sys.argv[2])

    lat_min = latitude;
    lon_min = longitude;

    lat_pixel_size = 2.7E-6
    lon_pixel_size = 2.7E-6
    ddims = 11
    bbox_str = str(lon_min-ddims*lon_pixel_size*1.0) + ',' + str(lat_min-ddims*lat_pixel_size*1.0) + ',' + str(lon_min+ddims*lon_pixel_size*1.0) + ',' + str(lat_min+ddims*lat_pixel_size*1.0)
    command = "curl --user Gophers:subregional.Varnished.1umbering 'https://services.digitalglobe.com/mapservice/wmsaccess?connectid=df282532-3390-42a8-b682-dab0abaa7b3a&SERVICE=WMS&REQUEST=GetFeatureInfo&version=1.1.1&SRS=EPSG:4326&BBOX=" + bbox_str + "&WIDTH=1000000&HEIGHT=100000&QUERY_LAYERS=DigitalGlobe:ImageryFootprint&X=5&Y=5&LAYERS=DigitalGlobe:ImageryFootprint&FEATURE_COUNT=100000'"
    # os.system(command)
    output = subprocess.check_output(command, shell=True)
    # print (output.splitlines())
    output = output.splitlines();
    matching = [s for s in output if b'acquisitionDate' in s]
    print ("No of images", len(matching))
    DGtimestamp = []
    for localimage in matching:
        DGtimestamp.append(localimage.split(b' ')[2].decode("utf-8"));
    return DGtimestamp;

In [14]:
time = getDigitalglobe(20,20)
print (time[0:2])

No of images 14
['2017-09-28', '2017-02-20']


In [ ]:
import plotly.graph_objs as go
import plotly
from datetime import datetime

plotly.offline.init_notebook_mode(connected=True)
data = [go.Scatter(x=time[0:2], y=[5,6])]
plotly.offline.iplot(data)

In [28]:
import numpy as np
NDVISTACK = np.load('/home/kumarv/tayal007/NDVISTACK_CLOUDFILTER.npy')
NDVIaverage = np.load('/home/kumarv/tayal007/NDVIaverage_CLOUDFILTER.npy')

In [4]:
def sortedtimestamp(directorypath):
   timestamps = dict()
   for path in directorypath:
        timestamps[datetime.datetime.strptime(path.split('.')[1][3:], '%y%j').date()] = path.split('.')[1]
   return timestamps;

directorypath = glob.glob("/panfs/roc/groups/6/kumarv/tayal007/MODIS_h20v08/*.hdf");
# print (directorypath)
sorteddates = sortedtimestamp(directorypath)
dates = []
for k, v in sorted(sorteddates.items()):
    dates.append(k);

In [ ]:
# plt.subplot(1, 2, 1)
from IPython.display import clear_output
plt.imshow(NDVIaverage)
while True:
    
    x = plt.ginput(1)
    clear_output()
#     print (NDVISTACK[int(round(x[0][1])),int(round(x[0][0])),:])
    try:
        lat = latitude[int(round(x[0][0]))][int(round(x[0][1]))]
        long = longitude[int(round(x[0][0]))][int(round(x[0][1]))]
        print ('http://maps.google.com/?q=' + str(lat) + ',' + str(long))
        showtimeseries(int(round(x[0][0])), int(round(x[0][1])));
    except IndexError:
        print ()
    
    
# plt.show()

In [5]:
def showtimeseries(yvalue, xvalue):  
  
#     plt.subplot(1, 2, 2).cla()
#     plt.subplot(1, 2, 2)
    counts = NDVISTACK[xvalue,yvalue,:]
    data = [go.Scatter(x=dates, y=counts)]
    plotly.offline.iplot(data)
#     pylab.figure(1)
#     axes = plt.gca()
#     axes.set_ylim([0,1])
#     x = range(len(names))
#     pylab.xticks(x, names)
#     plt.xticks( rotation=90,fontsize=5)
#     pylab.plot(x,counts,"g")
#     plt.plot(x,counts, 'ro')
#     plt.tight_layout()

In [76]:


trace0 = go.Scatter(
    x=[0,2, 3.5, 6],
    y=[0,1, 1.5, 1],
    mode='text',
)
data = [trace0]
layout = {
    'shapes': [
        {
            'type': 'line',
            'x0': 1,
            'y0': 0,
            'x1': 1,
            'y1': 2,
            'line': {
                'color': 'rgb(55, 128, 191)',
                'width': 3,
            },
        },
    ]
}

fig = {
    'data': data,
    'layout': layout,
}

plotly.offline.iplot(fig, filename='shapes-lines')

In [83]:
d = dict()
for x in range(2):
    print ("{ type: line, x0: " + time[x] + ", y0: 0, x1: " + time[x] + ", y1: 1, line: { color: rgb(55, 128, 191) , width: 3,},},")

{ type: line, x0: 2017-09-28, y0: 0, x1: 2017-09-28, y1: 1, line: { color: rgb(55, 128, 191) , width: 3,},},
{ type: line, x0: 2017-02-20, y0: 0, x1: 2017-02-20, y1: 1, line: { color: rgb(55, 128, 191) , width: 3,},},


In [89]:
x = "{\
            'type': 'line',\
            'x0': 1,\
            'y0': 0,\
            'x1': 1,\
            'y1': 2,\
            'line': {\
                'color': 'rgb(55, 128, 191)',\
                'width': 3,\
            },\
        },"
print (x)

{            'type': 'line',            'x0': 1,            'y0': 0,            'x1': 1,            'y1': 2,            'line': {                'color': 'rgb(55, 128, 191)',                'width': 3,            },        },


In [6]:
from __future__ import print_function
import glob
import numpy as np
import math
from pyhdf.SD import SD, SDC
import matplotlib
matplotlib.use('TkAgg')
import pylab
import matplotlib.pyplot as plt
import datetime
import subprocess
from numpy import genfromtxt
import plotly.graph_objs as go
import plotly
from scipy.spatial import cKDTree

In [11]:
def sortedtimestamp(directorypath):
	timestamps = dict()
	for path in directorypath:
            timestamps[datetime.datetime.strptime(path.split('.')[1][3:], '%y%j').date()] = path.split('.')[1]
	return timestamps;

In [12]:
directorypath = glob.glob("/panfs/roc/groups/6/kumarv/tayal007/MODIS_h20v08/*.hdf");
sorteddates = sortedtimestamp(directorypath)

In [14]:
import bs4
# load the file
with open("/home/kumarv/tayal007/plotly/name.html") as inf:
    txt = inf.read()
    soup = bs4.BeautifulSoup(txt)

/home/kumarv/tayal007/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning:

No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/kumarv/tayal007/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")




In [18]:
metatag = soup.new_tag('meta')
metatag.attrs['http-equiv'] = 'refresh'
metatag.attrs['content'] = '2'
soup.head.append(metatag)

In [19]:
# save the file again
with open("/home/kumarv/tayal007/plotly/name.html", "w") as outf:
    outf.write(str(soup))